# Entrenamiento modelo timeSformer para LSC

## Importar las dependencias necesarias

In [ ]:
from pathlib import Path

import torch
from transformers import Trainer, TrainingArguments, DefaultDataCollator, EarlyStoppingCallback
from datasets import DatasetDict, load_from_disk 

import config
# from train.optuna import objective
from data.load import load_video_metadata, create_label_mappings
from data.datasets import build_huggingface_dataset
from data.preprocesing import create_map_function
from model.load import load_model, load_image_processor
from training.metrics import compute_metrics

## Verificar si se cuenta con tarjeta grafica compatible con CUDA

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cuda



## PASO 1: Cargar Rutas y crear Metadata

In [ ]:
# Definir la ruta base
base_dataset_path = Path("../dataset")
# Cargar datos de cada split
# --- PASO 1: Cargar Metadatos ---
base_dataset_path = Path("../dataset")
video_metadata = load_video_metadata(base_dataset_path)

--- Cargando Metadatos desde: ..\dataset ---
  Procesando: ..\dataset\train...
    -> 150 videos cargados.
  Procesando: ..\dataset\eval...
    -> 30 videos cargados.
  Procesando: ..\dataset\test...
    -> 39 videos cargados.
Metadatos cargados para 219 videos.


## PASO 2: Crear Mapeos y construir DatasetDict

In [ ]:
label2id, id2label = create_label_mappings(video_metadata)
raw_dataset = build_huggingface_dataset(video_metadata, label2id)

print("\n--- Dataset Crudo Cargado ---")
print(raw_dataset)


--- Creando Mapeos de Etiquetas ---
Se crearon mapeos para 10 clases.
¡DatasetDict construido!

--- Dataset Crudo Cargado ---
DatasetDict({
    train: Dataset({
        features: ['video_path', 'label'],
        num_rows: 150
    })
    validation: Dataset({
        features: ['video_path', 'label'],
        num_rows: 30
    })
    test: Dataset({
        features: ['video_path', 'label'],
        num_rows: 39
    })
})


## PASO 3: Cargar Modelo e Image Processor

In [ ]:
model = load_model(
    model_ckpt=config.MODEL_CKPT,
    id2label=id2label,
    label2id=label2id,
    device=device,
    freeze_base=True,
    unfreeze_last_n=3
)
image_processor = load_image_processor(
    model_ckpt=config.PROCCESSOR
)

## PASO 4: Aplicando Preprocesamiento (PyAV + Aug Avanzada)

In [ ]:
height = width = image_processor.size["shortest_edge"]
resize_to = (height, width)
num_frames_to_sample = model.config.num_frames

processed_dataset_path = Path("./processed_sign_language_dataset")

if processed_dataset_path.exists():
    print(f"Cargando 'processed_dataset' desde el disco: {processed_dataset_path}")
    processed_dataset = load_from_disk(str(processed_dataset_path))
    # Es importante volver a establecer el formato después de cargar desde disco
    processed_dataset.set_format("torch")
    print("'processed_dataset' cargado.")
else:
    train_map_fn = create_map_function(image_processor, num_frames_to_sample, resize_to, is_train=True)
    val_test_map_fn = create_map_function(image_processor, num_frames_to_sample, resize_to, is_train=False)

    # Aplicar .map() - ESTO ES LO QUE CREA EL CONTENIDO DEL NUEVO DATASET
    processed_dataset = DatasetDict({
        'train': raw_dataset['train'].map(train_map_fn, remove_columns=['video_path'], batched=False), # Quitamos path y label_str
        'validation': raw_dataset['validation'].map(val_test_map_fn, remove_columns=['video_path'], batched=False),
        'test': raw_dataset['test'].map(val_test_map_fn, remove_columns=['video_path'], batched=False),
    })
    
    processed_dataset = processed_dataset.filter(
        lambda x: x['pixel_values'] is not None
    )
    print("Filtrado completado.")
    processed_dataset.save_to_disk(str(processed_dataset_path))
    # Establecer formato PyTorch
    processed_dataset.set_format("torch")

In [ ]:
import matplotlib.pyplot as plt

def show_video_frames(pixel_values, n=16, rows=4, cols=4):
    n = min(n, len(pixel_values))
    plt.figure(figsize=(cols*3, rows*3))
    for i in range(n):
        frame = pixel_values[i].permute(1, 2, 0).cpu().numpy()
        plt.subplot(rows, cols, i + 1)
        plt.imshow(frame)
        plt.axis('off')
        plt.title(f'Frame {i}')
    plt.tight_layout()
    plt.show()

# Ejemplo de uso:
example = processed_dataset['train'][0]
show_video_frames(example['pixel_values'], n=16, rows=4, cols=4)

## PASO 5: Configurar el Entrenamiento

In [ ]:
output_dir = "./results/timeSformer-LSC-finetuned"
learning_rate = 4.9610155578982295e-05 
batch_size = 4
num_epochs = 24 

training_args = TrainingArguments(
    output_dir=output_dir,
    learning_rate=learning_rate,
    per_device_train_batch_size=batch_size,
    per_device_eval_batch_size=batch_size,
    num_train_epochs=num_epochs,
    weight_decay=0.013320764183634398,         
    
    # Estrategias de Evaluación y Guardado
    eval_strategy="epoch",    # Evaluar al final de cada época
    save_strategy="epoch",          # Guardar al final de cada época
    load_best_model_at_end=True,    # Cargar el mejor modelo al final
    save_total_limit=2,             # Guardar solo los 2 últimos/mejores checkpoints
    metric_for_best_model="f1_weighted",
    greater_is_better=True,         # Mejor si el F1 ponderado es mayor
    # Logging y Otros
    logging_dir=f"{output_dir}/logs",
    logging_steps=10,               # Cada cuántos pasos mostrar logs
    remove_unused_columns=False,    # Importante si tu dataset tiene columnas extra
    fp16=torch.cuda.is_available(), # Usar precisión mixta si hay GPU (acelera y ahorra VRAM)
    # Early stopping
    # Opcional: Para Optuna más adelante
    # report_to="none", # Podrías cambiarlo a 'wandb' o 'tensorboard'
)

data_collator = DefaultDataCollator()

trainer = Trainer(
    model=model,             
    args=training_args,                
    train_dataset=processed_dataset["train"], 
    eval_dataset=processed_dataset["validation"], 
    tokenizer=image_processor,         
    compute_metrics=compute_metrics,
    data_collator=data_collator,   
    callbacks=[EarlyStoppingCallback(
            early_stopping_patience=3,        # <-- aquí va la paciencia
            early_stopping_threshold=0.0      # <-- aquí el delta mínimo de mejora
        )]
)

## PASO 6: Iniciando Entrenamiento

In [ ]:
try:
    # Lanza el proceso de entrenamiento
    train_results = trainer.train()

    print("\n--- Entrenamiento Finalizado ---")

    # Guarda el mejor modelo encontrado durante el entrenamiento
    print("Guardando el mejor modelo...")
    trainer.save_model()

    # Guarda las métricas del entrenamiento
    print("Guardando métricas de entrenamiento...")
    trainer.log_metrics("train", train_results.metrics)
    trainer.save_metrics("train", train_results.metrics)

    # Guarda el estado del Trainer (útil para reanudar)
    trainer.save_state()
    print("Métricas y estado del Trainer guardados.")

except Exception as e:
    print(f"\nOcurrió un error durante el entrenamiento: {e}")
    print("Revisa los mensajes de error anteriores.")
    print("Posibles causas: Memoria VRAM/RAM insuficiente (reduce 'batch_size'),")
    print("problemas con los datos, configuración incorrecta o errores de CUDA.")

## PASO 7: Testear el modelo

In [ ]:
from transformers import TimesformerForVideoClassification # El modelo que usas (ej: AutoModelForImageClassification)
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt

model = TimesformerForVideoClassification.from_pretrained(r"results\timeSformer-LSC-finetuned",)

# Prepara el Trainer solo para evaluación
test_trainer = Trainer(
    model=model,    
    args=training_args,  # Puedes reusar los mismos argumentos, ajustando output_dir si quieres
    tokenizer=image_processor,
    compute_metrics=compute_metrics,
    data_collator=data_collator,
    eval_dataset=processed_dataset["test"]
)

pred_output = test_trainer.predict(processed_dataset["test"])
y_true = pred_output.label_ids
y_pred = pred_output.predictions.argmax(axis=1)  # Si es clasificación multiclase


cm = confusion_matrix(y_true, y_pred)

class_names = [id2label[i] for i in range(len(id2label))]
disp = ConfusionMatrixDisplay(confusion_matrix=cm, display_labels=class_names)
disp.plot(cmap="Blues", xticks_rotation=45)
plt.title("Matriz de confusión en el test set")
plt.show()

In [ ]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, classification_report

# Accuracy
acc = accuracy_score(y_true, y_pred)
print(f"Accuracy: {acc:.4f}")

# Precision, Recall, F1 (macro y weighted)
precision_macro = precision_score(y_true, y_pred, average='macro', zero_division=0)
precision_weighted = precision_score(y_true, y_pred, average='weighted', zero_division=0)
recall_macro = recall_score(y_true, y_pred, average='macro', zero_division=0)
recall_weighted = recall_score(y_true, y_pred, average='weighted', zero_division=0)
f1_macro = f1_score(y_true, y_pred, average='macro', zero_division=0)
f1_weighted = f1_score(y_true, y_pred, average='weighted', zero_division=0)

print(f"Precision Macro:   {precision_macro:.4f}")
print(f"Precision Weighted:{precision_weighted:.4f}")
print(f"Recall Macro:      {recall_macro:.4f}")
print(f"Recall Weighted:   {recall_weighted:.4f}")
print(f"F1 Macro:          {f1_macro:.4f}")
print(f"F1 Weighted:       {f1_weighted:.4f}")

# Classification report con nombres de clase
target_names = [id2label[i] for i in range(len(id2label))]
print("\nClassification Report:")
print(classification_report(y_true, y_pred, target_names=target_names, zero_division=0))